In [1]:
import numpy as np
import pandas as pd
import os

In [10]:
data_directory = "../Data/Study/"
hololenses = ['A/', 'B/']
intermediate = ''
subdirectory = "recalibrated_ssHeadConstrained/"

In [3]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['Movement'] != "transition") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    #result = result[result['Movement'] != "transition"]
    #result = result[result['EyeTrackingDataValid'] == 'TRUE']
    #result = result[result['GazeInputSource'] == 'Eyes']
    result.reset_index(inplace=True, drop=True)
    return result

In [4]:
def calculate_cosine_error(df):
    df['cosineError'] = df.apply(lambda row: np.degrees(np.arccos(np.dot(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z'])), np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))/(np.linalg.norm(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z']))) * np.linalg.norm(np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))))), axis=1)
    return df

In [5]:
def calculate_euclidean_error(df):
    #df['euclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    #df['recalibratedEuclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['CalibratedGazeAngleX'], row['CalibratedGazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    df['euclideanError'] = np.sqrt((df.GazeAngleX-df.TargetAngleX) * (df.GazeAngleX-df.TargetAngleX) + (df.GazeAngleY-df.TargetAngleY) * (df.GazeAngleY-df.TargetAngleY))
    df['recalibratedEuclideanError'] = np.sqrt((df.CalibratedGazeAngleX-df.TargetAngleX) * (df.CalibratedGazeAngleX-df.TargetAngleX) + (df.CalibratedGazeAngleY-df.TargetAngleY) * (df.CalibratedGazeAngleY-df.TargetAngleY))
    return df

In [8]:
def calculate_errors(output_subdirectory):
    for hololens in hololenses:
        participantsList = os.listdir(data_directory + hololens)
        for participant in participantsList:
            print(participant)
            if not participant == '.DS_Store':
                if os.path.exists(data_directory + hololens + participant + '/' + intermediate + output_subdirectory) and len(os.listdir(data_directory + hololens + participant + '/' + intermediate + output_subdirectory)) == 12:
                    print('already calculated')
                    continue
                fileList = os.listdir(data_directory + hololens + participant + '/' + intermediate + subdirectory)
                for file in fileList:
                    if '.csv' in file:
                        filePath = data_directory + hololens + participant + '/' + intermediate
                        df = pd.read_csv(filePath + subdirectory + file)
                        error_df = calculate_euclidean_error(df)
                        error_df = calculate_cosine_error(error_df)
                        if not os.path.exists(filePath + output_subdirectory):
                            os.makedirs(filePath + output_subdirectory)
                        error_df_output_path = filePath + output_subdirectory + '/' + file
                        error_df.to_csv(error_df_output_path, index=False)
                        #print(file)
                        #print("uncalibrated: " + str(error_df['euclideanError'].mean()))
                        #print("calibrated: " + str(error_df['recalibratedEuclideanError'].mean()))
                        #print("cosine: " + str(error_df['cosineError'].mean()))
                        #print("")

In [11]:
calculate_errors("error/recalibrated_ssHeadConstrained")

03
ssWalking_20230505113947_shifted0.csv
uncalibrated: 64.87998304367781
calibrated: 56.25137220637573
cosine: 87.32342298037202

hallway_20230505114146_shifted0.csv
uncalibrated: 24.230997381199636
calibrated: 18.729398456057282
cosine: 23.28485264646472

ssHeadConstrained_20230505112732_shifted28.csv
uncalibrated: 11.82260662873523
calibrated: 3.786066553456924
cosine: 11.571138688339328

hallway_20230505112428_shifted0.csv
uncalibrated: 25.12340921302099
calibrated: 19.415298454224576
cosine: 23.99607746992986

wsBodyConstrained_20230505111519_shifted0.csv
uncalibrated: 28.90717282769959
calibrated: 25.076243558415943
cosine: 27.930655236679886

wsWalking_20230505111743_shifted27.csv
uncalibrated: 54.13070946299351
calibrated: 48.89165651342637
cosine: 87.74376542517335

calibration_20230505112000_shifted0.csv
uncalibrated: 7.406451449311706
calibrated: 4.3115823171049765
cosine: 7.314885230775771

ssHeadConstrained_20230505111305_shifted22.csv
uncalibrated: 5.6858374504165115
calib

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Study/A/43/recalibrated_ssHeadConstrained/'